In [2]:
!pip install chronos-forecasting
import pandas as pd  # requires: pip install pandas
import torch
from chronos import BaseChronosPipeline

  Using cached chronos_forecasting-1.5.0-py3-none-any.whl.metadata (30 kB)
  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
Using cached chronos_forecasting-1.5.0-py3-none-any.whl (29 kB)
Using cached transformers-4.47.1-py3-none-any.whl (10.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [4]:
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-bolt-small",
    device_map="cuda",  # use "cpu" for CPU inference and "mps" for Apple Silicon
    torch_dtype=torch.bfloat16,
)

df = pd.read_csv(
    "https://raw.githubusercontent.com/AileenNielsen/TimeSeriesAnalysisWithPython/master/data/AirPassengers.csv"
)

# context must be either a 1D tensor, a list of 1D tensors,
# or a left-padded 2D tensor with batch as the first dimension
# Chronos-Bolt models generate quantile forecasts, so forecast has shape
# [num_series, num_quantiles, prediction_length].
forecast = pipeline.predict(
    context=torch.tensor(df["#Passengers"]), prediction_length=12
)

AssertionError: Torch not compiled with CUDA enabled